# 벡터데이터 기반 RAG 어플리케이션 실습

🚀 목표는 뭘까?
RAG (Retrieval-Augmented Generation) 시스템을 만들기 위한 실습입니다.
즉, 웹에서 정보를 가져오고 → 그 내용을 기억하도록 정리(벡터화)한 다음 → 사용자 질문에 그 정보를 활용해 똑똑하게 대답하는 AI 챗봇을 만드는 겁니다.

📦 전체 흐름 요약




---


1.웹 페이지에서 정보 가져오기


---


마치 구글에서 웹문서를 복사해오는 것처럼, 웹에서 필요한 문서를 자동으로 긁어옵니다.

---
2.문서를 잘게 나누기 (텍스트 분할)

---



문서를 한 덩어리로 두면 AI가 이해하기 어려우니까, 문장을 일정 크기로 슬라이스해줍니다.
---
3.벡터 임베딩을 생성하고 저장하기
---
잘라낸 문장들을 숫자로 바꿔 기억하는 단계입니다.

즉, 문장을 **벡터(숫자 리스트)**로 바꿔 저장소에 저장합니다.
---
4.질문이 들어오면 관련 문장 찾기
---
사용자가 질문을 던지면, 아까 저장한 벡터 중 가장 관련 있는 문장을 찾아옵니다.
---
5.찾아온 문장 + 질문을 함께 LLM(GPT)에 전달하기
---
"이 문장들을 참고해서 이 질문에 답해줘!" 하고 GPT에게 건네주는 역할입니다.
---
6.GPT가 정답을 만들어냄
---
최종적으로 AI가 위 내용을 바탕으로 짧고 정확한 답변을 생성합니다.

🎯 쉽게 비유하자면...

 🔍 웹 문서는 책장에 꽂힌 책이고

 ✂️ 텍스트 분할은 책을 챕터별로 자르는 작업이고

 🧠 벡터화는 챕터를 AI가 이해할 수 있도록 기억시키는 것이고

 ❓ 사용자의 질문은 AI에게 "이 책에서 ~~내용 알려줘" 하는 것

 🤖 GPT는 책을 뒤져서 정확한 답을 말해주는 똑똑한 사서입니다.


단계	설명
---
1단계	웹 문서 긁어오기
---
2단계	텍스트 슬라이스(1000자씩 자르기)
---
3단계	잘린 텍스트 → 숫자(벡터)로 바꿔 저장
---
4단계	질문 → 관련된 텍스트 검색
---
5단계	질문 + 문맥 → GPT에 전달
---
6단계	GPT → 최종 응답 생성
---

💡 이런 걸 왜 쓰나요?
단순히 GPT에게 질문하면 모르는 척하거나 헛소리를 할 수 있습니다.

RAG는 GPT가 실제 자료를 참고해서 답하게 도와줍니다.

즉, GPT를 지식 기반 AI로 한 단계 업그레이드시키는 방법입니다!



In [ ]:
# 필요한 외부 라이브러리를 설치합니다.
# 포인트: 이 라이브러리들은 GPT와 외부 데이터를 연결하고 처리하는 데 꼭 필요합니다.
!pip install --upgrade openai langchain langchain-openai langchain-community beautifulsoup4 langchainhub -q
!pip install chromadb tiktoken -q

In [ ]:
# openai 모듈과 os 모듈을 불러옵니다. (os는 환경 변수 설정에 필요)
import openai
import os

# OpenAI API 키를 환경 변수로 등록합니다.
# 포인트: 이렇게 하면 코드에 키가 노출되지 않고 안전하게 사용할 수 있습니다.

# OpenAI API와 연결합니다.
client = openai.OpenAI()
print("API 키 설정이 완료되었습니다.")


API 키 설정이 완료되었습니다.


In [ ]:
# LangChain과 BeautifulSoup 관련 라이브러리 불러오기
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter  # 문서 자를 때 사용
from langchain_community.document_loaders import WebBaseLoader  # 웹 문서 로드
from langchain_community.vectorstores import Chroma  # 벡터 DB 저장소
from langchain_core.output_parsers import StrOutputParser  # 출력 형식 변환기
from langchain_core.runnables import RunnablePassthrough  # 입력 그대로 넘겨주는 모듈
from langchain_openai import ChatOpenAI, OpenAIEmbeddings  # GPT 모델 및 임베딩 생성
import bs4

# 웹에서 문서를 가져오는 WebBaseLoader를 설정합니다.
# 포인트: 지정한 웹 URL에서 특정 HTML 클래스에 해당하는 본문 내용만 추출하도록 합니다.
loader = WebBaseLoader(
    web_paths=("https://mlrx.tistory.com/entry/%EC%B1%97%EB%B4%87-PDF-QA-%EC%B1%97%EB%B4%87-%EA%B0%9C%EB%B0%9C%ED%95%98%EA%B8%B0-1",),

    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(class_="article_view")  # 본문 텍스트만 추출
    ),
)
docs = loader.load()


In [ ]:
docs  # 웹에서 가져온 문서 객체 확인

print(docs)  # 문서 내용 출력
print(type(docs))  # 문서의 데이터 타입 확인 (문서 분할 전에 데이터 형태 점검)

[Document(metadata={'source': 'https://mlrx.tistory.com/entry/%EC%B1%97%EB%B4%87-PDF-QA-%EC%B1%97%EB%B4%87-%EA%B0%9C%EB%B0%9C%ED%95%98%EA%B8%B0-1'}, page_content='\n\n\n목적\nPDF 문서 내용을 기반으로 질의응답(QA)를 할 수 있는 인트라넷에서 사용가능한 챗봇 개발\n\xa0\n준비물\npython\nlangchain\nopenai api key\n\xa0\n과정\n전체적인 플로우\n\n\n1. PDF 에서 텍스트 추출하기\nlangchain에서 제공하는 pdf loader를 이용해 pdf에서 text를 추출한다.\nlangchain에서는 다양한 방법을 제공하므로 각자 상황에 맞는 방법을 사용하도록 한다.\n(현재 글쓴이도 적절한 방법을 모색하고 있다.)\nfrom langchain.document_loaders import UnstructuredPDFLoader\nfrom langchain.text_splitter import RecursiveCharacterTextSplitter\n\nloader = UnstructuredPDFLoader(\'../약관.pdf\')\ndata = loader.load()\n\nprint(f"{len(data)}개의 문서를 가지고 있습니다.")\nprint(f"문서에 {len(data[0].page_content)}개의 단어를 가지고 있습니다.")\n1개의 문서를 가지고 있습니다.\n문서에 281012개의 단어를 가지고 있습니다.\n\xa0\n2. 텍스트를 chunk로 나누기\nembedding을 만들기 위해선 embedding이 모델이 소화할 수 있는 양만큼의 입력만 넣어줘야 한다. 그러기에 281012개의 단어를 가진 값을 통채로 넣어줄 수 없으므로 더 작은 단위로 묶어서 넣어주자.\ntext_splitter = RecursiveCharacterTextSplitter(chunk_size=

In [ ]:
# ✅ 2단계: 문서 텍스트 분할
# 긴 문서를 일정 크기로 잘라주는 작업입니다. GPT가 한 번에 처리할 수 있는 분량으로 조각냅니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# 실제로 문서를 분할합니다. -> splits 에는 잘린 문장 블록들이 담깁니다.
splits = text_splitter.split_documents(docs)
splits


In [ ]:
splits

In [ ]:
# ✅ 3단계: 텍스트를 임베딩(숫자 벡터)으로 변환하고 벡터 DB에 저장
# 포인트: GPT가 문장을 이해할 수 있도록 숫자로 바꾸고, 저장소에 기억시키는 단계입니다.
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 저장된 벡터로부터 검색 기능을 만드는 단계
retriever = vectorstore.as_retriever()


In [ ]:
# ✅ 4단계: 질문을 던지고, 관련된 문서를 검색
documents = retriever.get_relevant_documents("챗봇을 개발하기 위한 조건은?")

# 검색된 문서 출력
for document in documents:
    print(f"문서 내용: {document.page_content}")


In [ ]:
# ✅ 5단계: 기본 프롬프트 템플릿 정의
# GPT가 응답을 만들 때 따라야 할 지침을 정해줍니다.
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "당신은 질문에 답하는 작업을 수행하는 보조자입니다. "
        "다음에 제공된 문맥을 사용하여 질문에 답변하십시오. "
        "답을 모르면 모른다고 말하십시오. "
        "세 문장 이내로 답변하고 간결하게 작성하세요.\n"
        "질문: {question}\n"
        "문맥: {context}\n"
        "답변:"
    ),
)


In [ ]:

# ✅ 6단계: Hub에서 고급 프롬프트 불러오기 (선택사항)
# langchain hub에서 더 정교한 프롬프트를 불러옵니다.
# https://smith.langchain.com/hub/rlm/rag-prompt
# # https://smith.langchain.com/hub 에서 원하는 prompt를 받아올 수 있습니다.

from langchain import hub

# 프롬프트를 가져오기 위해 API 키 설
# os.environ["LANGSMITH_API_KEY"] = 

# 고급 프롬프트 가져오기 (더 정교한 지시를 포함)
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
# 프롬프트 내용 출력
print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [ ]:
# ✅ 7단계: LLM 설정 및 체인 구성
# 포인트: 질문과 문맥을 GPT에 함께 전달하고, 응답을 받는 전체 처리 흐름을 구성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0.1)

# 후처리 함수: 여러 문서를 하나의 문자열로 묶는 역할
def format_docs(docs):
    return "\n".join(doc.page_content for doc in docs)


In [ ]:
# 체인 구성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}  # 문맥을 검색하고 질문은 그대로 전달
    | prompt  # 문맥 + 질문 → GPT로 전달될 형태로 포맷
    | llm  # GPT가 응답 생성
    | StrOutputParser()  # 출력된 응답을 사람이 읽을 수 있는 텍스트로 정리
)

# ✅ 8단계: 질문에 대해 응답 생성
response = rag_chain.invoke("벡터 임베딩을 만들려면 어떻게 해야 하나요?")
print(response)  # GPT가 생성한 응답 출력

벡터 임베딩을 만들기 위해서는 먼저 텍스트를 모델이 처리할 수 있는 크기로 나누어야 합니다. 그런 다음, OpenAI의 text-embedding-ada-002와 같은 임베딩 모델을 사용하여 텍스트를 벡터로 변환합니다. 변환된 벡터는 FAISS와 같은 라이브러리를 사용하여 벡터 스토어에 저장할 수 있습니다.
